In [1]:
import tensorflow as tf
from transformers import BertTokenizer, TFBertModel
from keras.layers import Dense, Input, Dropout
from keras.models import Model

ImportError: cannot import name 'TFBertModel'

In [ ]:


# Load the pre-trained BERT model and tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-chinese')
bert_model = TFBertModel.from_pretrained('bert-base-chinese')

# Load the training data
train_data = [
  {'text': '你好', 'label': 'greeting'},
  {'text': '我要訂飯店', 'label': 'hotel_booking'},
  {'text': '請問附近有好吃的餐廳嗎？', 'label': 'restaurant_recommendation'}
  # add more training data here
]

# Tokenize the training data and convert labels to one-hot vectors
inputs = tokenizer([d['text'] for d in train_data], padding=True, truncation=True, max_length=128, return_tensors='tf')
labels = tf.one_hot([{'greeting': 0, 'hotel_booking': 1, 'restaurant_recommendation': 2}[d['label']] for d in train_data], depth=3)

# Build the model architecture
input_ids = Input(shape=(128,), dtype=tf.int32, name='input_ids')
attention_mask = Input(shape=(128,), dtype=tf.int32, name='attention_mask')
bert_output = bert_model({'input_ids': input_ids, 'attention_mask': attention_mask})[0]
dropout_output = Dropout(0.1)(bert_output)
dense_output = Dense(3, activation='softmax')(dropout_output)
model = Model(inputs=[input_ids, attention_mask], outputs=dense_output)

# Train the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit([inputs['input_ids'], inputs['attention_mask']], labels, epochs=3, batch_size=32)

# Test the model
test_data = [
  {'text': '你好嗎', 'label': 'greeting'},
  {'text': '我要預訂一晚房', 'label': 'hotel_booking'},
  {'text': '附近有沒有好吃的餐廳？', 'label': 'restaurant_recommendation'}
  # add more test data here
]
test_inputs = tokenizer([d['text'] for d in test_data], padding=True, truncation=True, max_length=128, return_tensors='tf')
test_labels = tf.one_hot([{'greeting': 0, 'hotel_booking': 1, 'restaurant_recommendation': 2}[d['label']] for d in test_data], depth=3)
test_loss, test_acc = model.evaluate([test_inputs['input_ids'], test_inputs['attention_mask']], test_labels)
print('Test loss:', test_loss)
print('Test accuracy:', test_acc)
